https://adventofcode.com/2019/day/20

In [27]:
import collections
import heapq
import networkx as nx

In [95]:
with open('data/20.txt') as fh:
    data = fh.read().rstrip('\n')

In [100]:
def load_points(data):
    D = {}
    for y, line in enumerate(data.split('\n')):
        for x, c in enumerate(line):
            if c == '.' or c.isalpha():
                D[(x, y)] = c
    return D

In [4]:
d1 = """\
         A           
         A           
  #######.#########  
  #######.........#  
  #######.#######.#  
  #######.#######.#  
  #######.#######.#  
  #####  B    ###.#  
BC...##  C    ###.#  
  ##.##       ###.#  
  ##...DE  F  ###.#  
  #####    G  ###.#  
  #########.#####.#  
DE..#######...###.#  
  #.#########.###.#  
FG..#########.....#  
  ###########.#####  
             Z       
             Z       
"""

In [5]:
p1 = load_points(d1)

In [6]:
def nsew(pt):
    x,y = pt
    return [(x, y-1), (x, y+1), (x+1, y), (x-1, y)]

In [7]:
def find_label(pt, D):
    if D[pt] != '.':
        return
    x, y = pt
    n = D.get((x, y-2), '9') + D.get((x, y-1), '9')
    if n.isalpha():
        return n
    s = D.get((x, y+1), '9') + D.get((x, y+2), '9')
    if s.isalpha():
        return s
    e = D.get((x+1, y), '9') + D.get((x+2, y), '9')
    if e.isalpha():
        return e
    w = D.get((x-2, y), '9') + D.get((x-1, y), '9')
    if w.isalpha():
        return w                          

In [8]:
def label_points(D):
    E = {}
    for pt, c in D.items():
        label = find_label(pt, D)
        if label:
            c = label
        E[pt] = c
    F = {}
    for pt, c in E.items():
        if c.isalpha() and len(c) == 1:
            pass
        else:
            F[pt] = c
    return F

In [9]:
p2 = label_points(p1)

In [44]:
p2

{(9, 2): 'AA',
 (9, 3): '.',
 (10, 3): '.',
 (11, 3): '.',
 (12, 3): '.',
 (13, 3): '.',
 (14, 3): '.',
 (15, 3): '.',
 (16, 3): '.',
 (17, 3): '.',
 (9, 4): '.',
 (17, 4): '.',
 (9, 5): '.',
 (17, 5): '.',
 (9, 6): 'BC',
 (17, 6): '.',
 (17, 7): '.',
 (2, 8): 'BC',
 (3, 8): '.',
 (4, 8): '.',
 (17, 8): '.',
 (4, 9): '.',
 (17, 9): '.',
 (4, 10): '.',
 (5, 10): '.',
 (6, 10): 'DE',
 (17, 10): '.',
 (17, 11): '.',
 (11, 12): 'FG',
 (17, 12): '.',
 (2, 13): 'DE',
 (3, 13): '.',
 (11, 13): '.',
 (12, 13): '.',
 (13, 13): '.',
 (17, 13): '.',
 (3, 14): '.',
 (13, 14): '.',
 (17, 14): '.',
 (2, 15): 'FG',
 (3, 15): '.',
 (13, 15): '.',
 (14, 15): '.',
 (15, 15): '.',
 (16, 15): '.',
 (17, 15): '.',
 (13, 16): 'ZZ'}

In [33]:
def points2graph(points):
    G = nx.Graph()
    portals = collections.defaultdict(list)
    for pt, c in points.items():
        if c.isalpha():
            portals[c].append(pt)
        G.add_node(pt, c=c)
        for nabe in nsew(pt):
            if nabe in points:
                G.add_node(nabe, c=points[nabe])
                G.add_edge(pt, nabe)
    for c, pts in portals.items():
        if len(pts) == 2:
            G.add_edge(*pts)
    return G    

In [34]:
g2 = points2graph(p2)

In [35]:
len(g2)

47

In [41]:
for nabe in g2.neighbors((9, 6)):
    print(nabe, g2.nodes[nabe]['c'])

(9, 5) .
(2, 8) BC


In [43]:
def run_maze(G):
    start = next(n for n in G if G.nodes[n]['c'] == 'AA')
    finish = next(n for n in G if G.nodes[n]['c'] == 'ZZ')
    return nx.shortest_path_length(G, start, finish)
    

In [44]:
run_maze(g2)

23

In [62]:
def bfs(G):
    start = next(n for n in G if G.nodes[n]['c'] == 'AA')
    finish = next(n for n in G if G.nodes[n]['c'] == 'ZZ')
    frontier = collections.deque() # Queue
    explored = set()
    frontier.append((0, start))
    explored.add(start)
    while frontier:
        pl, node = frontier.popleft()
        if node == finish:
            return pl
        for nabe in G.neighbors(node):
            if nabe not in explored:
                explored.add(nabe)
                frontier.append((pl+1, nabe))        

In [63]:
%%time
bfs(g2)

CPU times: user 134 µs, sys: 3 µs, total: 137 µs
Wall time: 140 µs


23

In [64]:
def dijkstra(G):
    start = next(n for n in G if G.nodes[n]['c'] == 'AA')
    finish = next(n for n in G if G.nodes[n]['c'] == 'ZZ')
    frontier = [] # heap
    explored = set()
    heapq.heappush(frontier, (0, start))
    explored.add(start)
    while frontier:
        pl, node = heapq.heappop(frontier)
        if node == finish:
            return pl
        for nabe in G.neighbors(node):
            if nabe not in explored:
                explored.add(nabe)
                heapq.heappush(frontier, (pl+1, nabe))
        

In [65]:
%%time
dijkstra(g2)

CPU times: user 113 µs, sys: 3 µs, total: 116 µs
Wall time: 119 µs


23

In [59]:
def make_graph(data):
    points = label_points(load_points(data))
    return points2graph(points)

In [60]:
run_maze(make_graph(d1))

23

In [61]:
%%time
run_maze(make_graph(data))

CPU times: user 79.4 ms, sys: 0 ns, total: 79.4 ms
Wall time: 78.4 ms


484

## Part 2

In [82]:
def points2rgraph(points):
    G = nx.Graph()
    for pt, c in points.items():
        G.add_node(pt, c=c)
        for nabe in nsew(pt):
            if nabe in points:
                G.add_node(nabe, c=points[nabe])
                G.add_edge(pt, nabe)
    return G    

In [89]:
def rdijksra(G):
    start = next(n for n in G if G.nodes[n]['c'] == 'AA')
    finish = next(n for n in G if G.nodes[n]['c'] == 'ZZ')
    xmin = min(x for (x,y) in G)
    xmax = max(x for (x,y) in G)
    ymin = min(y for (x,y) in G)
    ymax = max(y for (x,y) in G)

    portals = collections.defaultdict(list)
    for n in G:
        c = G.nodes[n]['c']
        if c.isalpha() and c not in ('AA', 'ZZ'):
            portals[c].append(n)
    for (a, b) in portals.values():
        ax, ay = a
        dl = 1 if ax in {xmin, xmax} or ay in {ymin, ymax} else -1
        G.nodes[a]['dl'] = dl
        G.nodes[a]['p'] = b
        bx, by = b
        dl = 1 if bx in {xmin, xmax} or by in {ymin, ymax} else -1
        G.nodes[b]['dl'] = dl
        G.nodes[b]['p'] = a
    
    frontier = [] # heap
    explored = set()
    heapq.heappush(frontier, (0, 0, start))
    explored.add((start, 0))
    
    while frontier:
        level, steps, node = heapq.heappop(frontier)
        if node == finish and level == 0:
            return steps
        for nabe in G.neighbors(node):
            if (nabe, level) not in explored:
                explored.add((nabe, level))
                heapq.heappush(frontier, (level, steps+1, nabe))
        if 'p' in G.nodes[node]:
            portal = G.nodes[node]['p']
            dl = G.nodes[portal]['dl']
            nextlevel = level + dl
            if nextlevel >= 0 and (portal, nextlevel) not in explored:
                explored.add((portal, nextlevel))
                heapq.heappush(frontier, (nextlevel, steps+1, portal))

In [79]:
d1 = """\
         A           
         A           
  #######.#########  
  #######.........#  
  #######.#######.#  
  #######.#######.#  
  #######.#######.#  
  #####  B    ###.#  
BC...##  C    ###.#  
  ##.##       ###.#  
  ##...DE  F  ###.#  
  #####    G  ###.#  
  #########.#####.#  
DE..#######...###.#  
  #.#########.###.#  
FG..#########.....#  
  ###########.#####  
             Z       
             Z       
"""

In [80]:
points = label_points(load_points(d1))

In [83]:
rg1 = points2rgraph(points)

In [90]:
rdijksra(rg1)

26

In [91]:
d2 = """\
             Z L X W       C                 
             Z P Q B       K                 
  ###########.#.#.#.#######.###############  
  #...#.......#.#.......#.#.......#.#.#...#  
  ###.#.#.#.#.#.#.#.###.#.#.#######.#.#.###  
  #.#...#.#.#...#.#.#...#...#...#.#.......#  
  #.###.#######.###.###.#.###.###.#.#######  
  #...#.......#.#...#...#.............#...#  
  #.#########.#######.#.#######.#######.###  
  #...#.#    F       R I       Z    #.#.#.#  
  #.###.#    D       E C       H    #.#.#.#  
  #.#...#                           #...#.#  
  #.###.#                           #.###.#  
  #.#....OA                       WB..#.#..ZH
  #.###.#                           #.#.#.#  
CJ......#                           #.....#  
  #######                           #######  
  #.#....CK                         #......IC
  #.###.#                           #.###.#  
  #.....#                           #...#.#  
  ###.###                           #.#.#.#  
XF....#.#                         RF..#.#.#  
  #####.#                           #######  
  #......CJ                       NM..#...#  
  ###.#.#                           #.###.#  
RE....#.#                           #......RF
  ###.###        X   X       L      #.#.#.#  
  #.....#        F   Q       P      #.#.#.#  
  ###.###########.###.#######.#########.###  
  #.....#...#.....#.......#...#.....#.#...#  
  #####.#.###.#######.#######.###.###.#.#.#  
  #.......#.......#.#.#.#.#...#...#...#.#.#  
  #####.###.#####.#.#.#.#.###.###.#.###.###  
  #.......#.....#.#...#...............#...#  
  #############.#.#.###.###################  
               A O F   N                     
               A A D   M                     
"""

In [92]:
points = label_points(load_points(d2))

In [93]:
rg2 = points2rgraph(points)

In [94]:
%%time
rdijksra(rg2)

CPU times: user 3.37 ms, sys: 0 ns, total: 3.37 ms
Wall time: 3.4 ms


396

In [96]:
d3 = data

In [97]:
points = label_points(load_points(d3))

In [98]:
rg3 = points2rgraph(points)

In [99]:
%%time
rdijksra(rg3)

CPU times: user 202 ms, sys: 0 ns, total: 202 ms
Wall time: 200 ms


5754